In [1]:
import os

In [2]:
%pwd

'd:\\projects\\Kidney-Disease-Classification-Deep-Learning-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\projects\\Kidney-Disease-Classification-Deep-Learning-Project'

In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\projects\\Kidney-Disease-Classification-Deep-Learning-Project'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\projects\\Kidney-Disease-Classification-Deep-Learning-Project'

In [5]:
%pwd

'd:\\projects\\Kidney-Disease-Classification-Deep-Learning-Project'

In [4]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/rohitdhotare2001/Kidney-Disease-Classification-Deep-Learning-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="rohitdhotare2001"
os.environ["MLFLOW_TRACKING_PASSWORD"]="Rohit@1745"

In [5]:
import tensorflow as tf

In [6]:
model=tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_url: str
    params_image_size:list
    params_batch_size: int

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json


In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_url="https://dagshub.com/rohitdhotare2001/Kidney-Disease-Classification-Deep-Learning-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [10]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [11]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_url)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [12]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-12-02 11:58:41,103: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-02 11:58:41,112: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-02 11:58:41,117: INFO: common: created directory at: artifacts]
Found 300 images belonging to 2 classes.
19/19 [==============================] - 34s 2s/step - loss: 40.5313 - accuracy: 0.5000
[2024-12-02 11:59:16,048: INFO: common: json file saved at: scores.json]


2024/12/02 11:59:19 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-12-02 11:59:22,252: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\rohit\AppData\Local\Temp\tmpol0mva2f\model\data\model\assets
[2024-12-02 11:59:23,395: INFO: builder_impl: Assets written to: C:\Users\rohit\AppData\Local\Temp\tmpol0mva2f\model\data\model\assets]


2024/12/02 11:59:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/12/02 12:00:31 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 5
Created version '5' of model 'VGG16Model'.


🏃 View run indecisive-chimp-371 at: https://dagshub.com/rohitdhotare2001/Kidney-Disease-Classification-Deep-Learning-Project.mlflow/#/experiments/0/runs/787d21207ae44102ac07f7858b090d89
🧪 View experiment at: https://dagshub.com/rohitdhotare2001/Kidney-Disease-Classification-Deep-Learning-Project.mlflow/#/experiments/0
